# Collaboration and Competition - Report

![tennis](tennis_image.png)

In this project, I trained two Deep Deterministic Policy Gradient (DDPG) agents to solve the Unity Tennis environment. A description of the environment can be found in the project **Readme**. I will explain the low level details of the agents and learning algorithm in this report.

For this multi-agent training task, each agent must learn how to strike the ball such that it passes over the net. My approach is to treat this as a self-play task, with the agents sharing information and learning from each other's experiences. **The agents have their own local observations and actions, but share the actors, critics and the experience buffer**. First, let us look into implementation of the DDPG algorithm.

A DDPG agent approximates the long-term reward, given states and actions, using a critic value function representation. The major pieces in a DDPG agent are: 

* Deterministic actor (or policy)
* Critic that outputs the Q-Value function Q(s,a)
* Buffer that stores experiences in offline fashion
* Noise model that aids in exploration (we will use OU noise).

### Deterministic actor

To model the deterministic actor, I used a neural network with two hidden layers of size 256 and 128. The inputs to the neural network are the **local observations** of each agent (24) and the outputs are the **local actions** (2). A **Tanh layer** wraps the final Linear layer to ensure that the output remains within the bounds -1 and 1.

A representation of the actor layers is shown below:

                                  ReLU               ReLU             Tanh
**INPUT S (24)**  ------------> **LINEAR (256)**  ------------ **LINEAR (128)**  ------------ **LINEAR (2)** ------------> **OUTPUT A (2)**

### Critic
The critic takes states as inputs and outputs the state-action value Q(s,a). I use a similar neural network structure as the actor to model the critic. A representation of the critic layers is shown below:

                                   ReLU               ReLU             ReLU
**INPUT S(24), A(2)**  ------------ **LINEAR (256)**  ------------ **LINEAR (128)**  ------------ **LINEAR (1)**   ------------ **OUTPUT Q(S,A) (1)**

### Experience Buffer

DDPG agents train from an offline experience buffer that stores experiences as the agents interact with the environment. The experiences are states (S), actions (A), rewards (R), next states (NS) and done (D) values. 

To model the experience buffer, I use a **deque** data structure with a **maximum length of 1e5**. The buffer generates uniformly sampled experiences which are then used to update the actor and critic networks during training.

### Exploration Noise

To boost exploration during training, I modeled the action noise based on **Ornstein-Uhlenbeck action noise**. The key parts of the noise are the mean attraction constant (MAC), variance (VAR), and decay rate (DECAY). Tuning the noise properties had a big impact on the training. The noise dynamics follows the relationship:

In [2]:
noise = prev_noise + MAC * (MEAN - prev_noise) + VAR * np.random.randn

**The noise mean was set to 0 with mean attraction constant set to 0.13 and variance set to 0.2.** The variance decays at the rate of 1e-5 until a minimum threshold of 0.001 is reached. These noise parameters were crafted carefully so that they do not introduce too much exploration as well as decay to near zero after 800 episodes.

### Learning algorithm

DDPG is a cross between value-based and policy gradient algorithms. The Q-Learning side of DDPG approximates the max Q-value of a state-action pair. The policy gradient side of DDPG uses the Q-function to learn the policy. The high level steps in the training algorithm is as follows:

1. Sample experiences from the experience buffer.
2. Train the critic by minimizing the TD-errors through stochastic gradient descent.
3. Train the actor by maximizing the Q-function through gradient ascent.
4. The algorithm also uses target networks for the actor and critic which are their time-delayed copies. These target networks are updated at regular intervals and greatly stabilize the training process.

In [ ]:
def learn(self, experiences):
        """
        Train the actor and critic.
        
        """
        
        # unpack experience
        states, actions, rewards, next_states, dones = experiences
        
        # compute td targets
        with torch.no_grad():
            target_action = self.target_actor.mu(next_states)
            targetQ = self.target_critic.Q(next_states,target_action)
            y = rewards + self.gamma * targetQ * (1-dones)
        
        # compute local Q values
        Q = self.critic.Q(states, actions)
        
        # critic loss
        critic_loss = F.mse_loss(Q,y)

        # update critic
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.critic.parameters(), 1)  # gradient clipping
        self.critic_optimizer.step()
        
        # freeze critic before policy loss computation
        for p in self.critic.parameters():
            p.requires_grad = False
        
        # actor loss
        actor_loss = -self.critic.Q(states, self.actor.mu(states)).mean()
        
        # update actor
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.actor.parameters(), 1)  # gradient clipping
        self.actor_optimizer.step()
        
        # Unfreeze critic
        for p in self.critic.parameters():
            p.requires_grad = True
            
        # log the loss and noise
        self.actor_loss_log.append(actor_loss.detach().cpu().numpy())
        self.critic_loss_log.append(critic_loss.detach().cpu().numpy())
        #self.noise_log.append(np.mean(self.noise_model.x))
        
        # soft update target actor and critic
        if self.step_count % self.update_freq == 0:
            self.soft_update(self.target_actor, self.actor)
            self.soft_update(self.target_critic, self.critic)
            
    
    def soft_update(self, target_model, model):
        """
        Soft update target networks.
        """
        with torch.no_grad():
            for target_params, params in zip(target_model.parameters(), model.parameters()):
                target_params.data.copy_(self.tau*params + (1-self.tau)*target_params.data)
    

### Hyperparameters

I used following hyperparameters for the DDPG agent:

1. An experience buffer length of 1e5, which is sufficient to store a large set of experiences.
2. Actor and critic learn rates of 1e-4 and 2e-4 respectively. The critic learns a bit faster than the actor since it guides the actor updates.
3. A target network smoothing factor of 0.05. I found that a higher smoothing factor better stabilized the training.
4. A discount factor of 0.995. This encouraged long term rewards.

### Final pieces

Initially, the training algorithm from the [original DDPG paper](https://arxiv.org/pdf/1509.02971.pdf) did not do well even after extensive tuning of hyperparameters (a few weeks of my life were lost). The following additions greatly improved the training performance:

1. Train every **n** episodes instead of every episode. I found the magic number to be 5! The agents just collect experiences when not training.
2. Updating the actor and critic networks in multiple iterations. In the final implementation I update the networks 10 times (from newly sampled experiences) every training iteration.

I found some of these techniques also used in popular RL repositories like [SpinningUp](https://github.com/openai/spinningup) and [RLLib](https://docs.ray.io/en/latest/rllib.html).

### Results

With the above approach and implementation, the agents solved the environment in **675 episodes**. The following plot shows that the agents learn sufficiently to reach the average reward of +0.5. I continued the training even after the desired reward was reached to see where it peaks. In most training attempts, the reward would peak at 0.8-0.9 and then drop drastically. However, in this plot I show the progress to around +0.6 only.

![Image](training_results.png)

***Note:***
The above training was performed on a GPU (Udacity workspace). I observed that CPU training convergence was better than GPU, with CPU taking fewer episodes to reach +0.5 (although it took more wall clock time). In one session, the environment was solved in as less as ***480 episodes*** (see **Tennis_cpu.ipynb**). I recommend using CPU if you are reproducing results with my code.

### Future Work

The approach to train the agent was based on sharing key parts like the actor, critic and experience buffer between the agents. Another approach could be using centralized critics and decentralized actors for training, similar to the work done on [MADDPG](https://arxiv.org/abs/1706.02275).

There is an implementation of MADDPG in my repositiory under the name **tennis_train_maddpg.py** but it did not work well during training. Further work needs to be done to tune hyperparameters for this implementation. Feel free to check out the MADDPG code and create an issue in Github if you have suggestions.

DDPG algorithms can benefit from the use of Prioritized Experience Replay (PER) which stores experience priorities and samples data based on those priorities. PER has been shown to greatly improve performance in offline RL algorithms. It will be interesting to see how much a PER can improve training for this environment.

Finally, TD3 and SAC algorithms are improvements on DDPG and may be of interest.